In [ ]:
! pip install -U \
  langchain \
  langchain-openai \
  langchain-community \
  langchain-text-splitters \
  faiss-cpu \
  pypdf

In [3]:
# ==========================================================
# Document Question Answering (RAG) with Memory
# LangChain 0.2+ | LCEL | FAISS | OpenAI
# ==========================================================

# -----------------------------
# 1. Imports (LATEST & STABLE)
# -----------------------------
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_community.chat_message_histories import ChatMessageHistory

from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory

# -----------------------------
# 2. Load LLM
# -----------------------------
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

# -----------------------------
# 3. Load & split document
# -----------------------------

file_apth = "./transformers.pdf"
loader = PyPDFLoader(file_apth)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

chunks = text_splitter.split_documents(documents)

# -----------------------------
# 4. Create vector store
# -----------------------------
embeddings = OpenAIEmbeddings()

vectorstore = FAISS.from_documents(
    chunks,
    embedding=embeddings
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# -----------------------------
# 5. Helper: format docs
# -----------------------------
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# -----------------------------
# 6. Prompt with chat history
# -----------------------------
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a helpful assistant. Answer ONLY using the provided context. "
        "If the answer is not in the context, say 'I don't know'."
    ),
    MessagesPlaceholder(variable_name="history"),
    (
        "human",
        "Context:\n{context}\n\nQuestion:\n{question}"
    )
])

# -----------------------------
# 7. RAG chain (LCEL)
# -----------------------------
rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)

# -----------------------------
# 8. Chat memory
# -----------------------------
message_history = ChatMessageHistory()

rag_chain_with_memory = RunnableWithMessageHistory(
    rag_chain,
    lambda session_id: message_history,
    input_messages_key="question",
    history_messages_key="history",
)

# -----------------------------
# 9. Chat loop
# -----------------------------
print("\n📄 Document QA system is ready!")
print("Type 'exit' to quit.\n")

while True:
    question = input("Ask a question: ")

    if question.lower() == "exit":
        break

    response = rag_chain_with_memory.invoke(
        question,
        config={"configurable": {"session_id": "student-demo"}}
    )

    print("\nAnswer:")
    print(response.content)
    print("-" * 50)


📄 Document QA system is ready!
Type 'exit' to quit.



KeyboardInterrupt: Interrupted by user